## III c

In [50]:
import torch.nn as nn

class TrivialPhi(nn.Module):
    """ Trivial implementation of Phi"""
    def forward(self,inputs):
        return inputs
    
class DensePhi(nn.Module):
    """ Simple dense layer phi"""
    def __init__(self,phi_in,phi_out):
        super().__init__()
        self.layer=nn.Linear(in_features=phi_in,out_features=phi_out)
        self.reLU=nn.ReLU()
    
    def forward(self,inputs):
        return self.reLU(self.layer(inputs))

In [51]:
import torch 

class F(nn.Module):
    def __init__(self,phi_in:int,phi:nn.Module): 
        """Creates a new instance of F.
        Parameters
        - - - - - -
        phi_in : int
        The input dimension of phi, i.e., the expected
        shape is [batch_size, phi_in]
        phi : nn.Module
        PyTorch module which implements a neural network block "Phi"
        """
        super().__init__()
        self.phi = phi
        self.phi_in = phi_in
        self.xb_size = phi_in[1]
        
    def forward(self, inputs):
        xa,xb=inputs.split([inputs.shape[1]-self.xb_size,self.xb_size],dim=1)
        ya = xa + self.phi(xb)
        yb = xb
        return torch.cat((ya,yb),axis=1)

    def inverse(self, inputs):
        ya,yb=inputs.split([inputs.shape[1]-self.xb_size,self.xb_size],dim=1)
        xb=yb
        xa=ya-self.phi(yb)
        return torch.cat((xa,xb),axis=1)


In [10]:
import unittest

def test_phi(x):
    return x

class TestF(unittest.TestCase):
    def setUp(self):
        phi_in = (3,2) # batch_size = 3
        phi_module=TrivialPhi()
        self.model=F(phi_in,phi_module)
        xa = torch.tensor(((1,2),(1,2),(1,2)))
        xb = torch.tensor(((3,4),(3,4),(3,4)))
        self.x = torch.cat((xa,xb),axis=1)
        ya = torch.tensor(((4,6),(4,6),(4,6)))
        yb = torch.tensor(((3,4),(3,4),(3,4)))
        self.y= torch.cat((ya,yb),axis=1)
        
    def test_forward(self):
        y_pred = self.model(self.x)
        self.assertTrue(torch.equal(y_pred,self.y))
        
    def test_inverse(self):
        x_pred = self.model.inverse(self.y)
        self.assertTrue(torch.equal(x_pred,self.x))
            
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.034s

OK


## III d

In [12]:
import torch
import torch.nn as nn

class S(nn.Module):
    def __init__(self, n_A, n_B):
        """ Creates a new instance of S.
        Parameters
        - - - - - -
        n_A : int
        Number of components for x_A
        n_B : int
        Number of components for x_B
        """
        
        super().__init__()
        self.n_A=n_A
        self.n_B=n_B
    
    def forward(self, inputs):
        xa,xb=inputs.split([self.n_A,self.n_B],dim=1)
        return torch.cat((xb,xa),dim=1)
    
    def inverse(self, inputs):
        xb,xa=inputs.split([self.n_B,self.n_A],dim=1)
        return torch.cat((xa,xb),dim=1)


In [13]:
import unittest

class TestS(unittest.TestCase):
    def setUp(self):
        self.model=S(2,2)
        # batch size 4
        xa = torch.tensor(((1,2),(1,2),(1,2),(1,2)))
        xb = torch.tensor(((3,4),(3,4),(3,4),(3,4)))
        self.forward=torch.cat(tensors=(xa,xb),dim=1)
        self.backward=torch.cat(tensors=(xb,xa),dim=1)
        
    def test_forward(self):
        y_pred = self.model(self.forward)
        self.assertTrue(torch.equal(y_pred,self.backward))
        
    def test_inverse(self):
        y_pred = self.model.inverse(self.backward)
        self.assertTrue(torch.equal(y_pred,self.forward))
                    
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok
test_forward (__main__.TestS) ... ok
test_inverse (__main__.TestS) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.012s

OK


## III e 

In [45]:
class G(nn.Module):
    def __init__(self,n_A,n_B,phi):
        """ Creates a new instance of G.
        Parameters
        - - - - - -
        n_A : int
        Number of components for x_A
        n_B : int
        Number of components for x_B
        """
        
        super().__init__()
        self.n_A=n_A
        self.n_B=n_B
        
        self.F1=F(phi_in=(4,2),phi=phi)
        self.S1=S(2,2)
        self.F2=F(phi_in=(4,2),phi=phi)
        self.S2=S(2,2)
        self.F3=F(phi_in=(4,2),phi=phi)
        self.network_stack=[self.F1,self.S1,self.F2,self.S2,self.F3]
        
    def forward(self,inputs):
        current_value=inputs
        for layer in self.network_stack:
            current_value=layer(current_value)
        return current_value
    
    def inverse(self,inputs):
        current_value=inputs
        for layer in self.network_stack:
            current_value=layer.inverse(current_value)
        return current_value
    

In [65]:
import unittest
import torch.optim as optim

class TestG(unittest.TestCase):
    def setUp(self):
        self.model=G(2,2,TrivialPhi())
        # batch size 4
        xa = torch.tensor(((1,2),(1,2),(1,2),(1,2))).float()
        xb = torch.tensor(((3,4),(3,4),(3,4),(3,4))).float()
        ## calculated this solution by hand
        ya = torch.tensor(((11,16),(11,16),(11,16),(11,16))).float()
        yb = torch.tensor(((7,10),(7,10),(7,10),(7,10))).float()
        self.input=torch.cat(tensors=(xa,xb),dim=1)
        self.solution=torch.cat(tensors=(ya,yb),dim=1)
        
    def test_forward(self):
        y_pred = self.model(self.input)
        self.assertTrue(torch.equal(y_pred,self.solution))
        
    def test_inverse(self):
        y_pred = self.model.inverse(self.solution)
        self.assertTrue(torch.equal(y_pred,self.input))
        
    def test_training(self):
        model= G(2,2,DensePhi(phi_in=2,phi_out=2))
        loss_fn = nn.MSELoss()
        optimizer=optim.Adam(model.parameters(), lr=0.005)
        
        #training 
        for epoch in range(0,1000):
            optimizer.zero_grad()
            outputs = model(self.input)
            loss = loss_fn(outputs, self.solution)
            loss.backward()
            optimizer.step()
        
        #test
        with torch.no_grad():
            y_pred = model(self.input)
            print(y_pred)
            print(self.solution)
            relative_tolerance=0.1
            absolute_tolerance=1
            elementwise_compare = torch.isclose(y_pred,self.solution,relative_tolerance,absolute_tolerance)
            self.assertTrue(elementwise_compare.all())
        
                    
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok
test_forward (__main__.TestG) ... ok
test_inverse (__main__.TestG) ... ok
test_training (__main__.TestG) ... ok
test_forward (__main__.TestS) ... ok
test_inverse (__main__.TestS) ... 

tensor([[10.9411, 16.0069,  7.1359,  9.9806],
        [10.9411, 16.0069,  7.1359,  9.9806],
        [10.9411, 16.0069,  7.1359,  9.9806],
        [10.9411, 16.0069,  7.1359,  9.9806]])
tensor([[11., 16.,  7., 10.],
        [11., 16.,  7., 10.],
        [11., 16.,  7., 10.],
        [11., 16.,  7., 10.]])


ok

----------------------------------------------------------------------
Ran 7 tests in 1.463s

OK
